In [1]:
import numpy as np
from sklearn.svm import SVC
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler

In [2]:
"""
this is the code you need to run to import the data. 
You just have to change line 40 putting the correct path.
"""

train = pd.read_csv('fashion-mnist_train.csv')
test = pd.read_csv('fashion-mnist_test.csv')

indexes = [2,3]

train_set = train[train['label'].isin(indexes)]
test_set = test[test['label'].isin(indexes)]

X_Train = train_set.values[:,1:]
Y_Train = train_set.values[:,0]

X_Test = test_set.values[:,1:]
Y_Test = test_set.values[:,0]

ind_2 = np.where(Y_Train==2)
ind_2_Test = np.where(Y_Test==2)

ind_3 = np.where(Y_Train==3)
ind_3_Test = np.where(Y_Test==3)

X2 = X_Train[ind_2[0][:1000]]
Y2 = np.ones(1000)

X3 = X_Train[ind_3[0][:1000]]
Y3 = -np.ones(1000)

X2test = X_Test[ind_2_Test[0][:200]]
Y2test = np.ones(X2test.shape[0])

X3test = X_Test[ind_3_Test[0][:200]]
Y3test = -np.ones(X3test.shape[0])


X_Train = np.concatenate((X2,X3))
Y_Train = np.concatenate((Y2,Y3))


X_Test = np.concatenate((X2test,X3test))
Y_Test = np.concatenate((Y2test,Y3test))

In [3]:
Y_Train = Y_Train.astype(int)

In [4]:
# Check oversampling
k = 0
j = 0
for i in Y_Train:
    if i == 1:
        k += 1
    else:
        j += 1
print(k)
print(j)

1000
1000


In [5]:
def scale_dataset(X, y, oversample=False):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
     
    if oversample:
        ros = RandomOverSampler()
        X, y = ros.fit_resample(X, y)

    data = np.hstack((X, np.reshape(y, (len(y), 1))))
    
    return data, X, y

In [6]:
train, x_train, y_train = scale_dataset(X_Train, Y_Train)
test, x_test, y_test = scale_dataset(X_Test, Y_Test)

In [7]:
y_train = y_train.astype(float)
y_test = y_test.astype(float)

In [8]:
# K_fold cross validation
def k_fold_cross_validation(data, num_folds):
    fold_size = len(data) // num_folds
    fold_indices = np.arange(len(data))
    np.random.seed(2108602)
    np.random.shuffle(fold_indices)

    folds_data = {}

    for i in range(num_folds):
        validation_indices = fold_indices[i * fold_size : (i + 1) * fold_size]
        train_indices = np.concatenate([fold_indices[:i * fold_size], fold_indices[(i + 1) * fold_size:]])

        folds_data[i] = {
            'train': data[train_indices],
            'validation': data[validation_indices]
        }

    return folds_data

In [9]:
n_folds = 5
folds = k_fold_cross_validation(train, n_folds)

In [10]:
for i in np.arange(len(folds)):
    print("Fold number: ", i)
    k_t = folds[i]['train']
    k_v = folds[i]['validation']
    x_train = k_t[:, :-1]
    y_train = k_t[:, -1]
    x_valid = k_v[:, :-1]
    y_valid = k_v[:, -1]
    svm_model = SVC()
    svm_model.fit(x_train, y_train)
    y_hat = svm_model.predict(x_valid)
    accuracy = np.sum(y_valid == y_hat) / len(y_valid)
    print(f"Fold: {i}, Acc.: {accuracy}")

Fold number:  0
Fold: 0, Acc.: 0.9775
Fold number:  1
Fold: 1, Acc.: 0.97
Fold number:  2
Fold: 2, Acc.: 0.96
Fold number:  3
Fold: 3, Acc.: 0.985
Fold number:  4
Fold: 4, Acc.: 0.9775


In [11]:
k_t = folds[3]['train']
k_v = folds[3]['validation']
x_train = k_t[:, :-1]
y_train = k_t[:, -1]
best_model = svm_model.fit(x_train, y_train)
y_hat = svm_model.predict(x_test)
accuracy = np.sum(y_test == y_hat) / len(y_test)
print(accuracy)


0.975
